# Predicting Yelp Rating from Health Inspection Scores by Restaurant

The first analysis point that the team agreed upon was to see if there is a correlation between the Boulder County Health Inspection Scores and Yelp or Google Ratings for restaurants in Boulder county, Colorado.

The Boulder County Health Inspections Scores were obtained. The features selected for the first analysis were the Health Inspection Score, Facility Type and Facility Category. These features were used to train the model in trying to predict the Yelp Rating per facility. 

The first step in engineering the features for the machine learning model used the filtered dataset to:
* Eliminate all location data so as not to overburden the model
* Average the inspection scores for all routine and regular health inspections by facility
    * This was difficult to eliminate the duplicate rows without losing details (pivot table and merge)
* Bin the averaged health inspection scores to match the Health Department ratings
* Create randomized Yelp Ratings to test the model
* Use Random Forest model as it is fast, simple and flexible 
    * Easy to use during the initial model development process, to see how it performs
    * Provides a good indicator of the importance it assigns to features
    * Limitations include: fast to train, but quite slow to create predictions once they are trained
    
* May need to switch to a neural network, for the second phase which has a lot of different feature types

In [7]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import tensorflow as tf

# Import our input datasets
ave_score_df=pd.read_csv('db_average_health_scores_bucketed.csv', index_col=0)
ave_score_df.drop("ave_insp_score", axis=1, inplace=True)
ave_score_df

,facilityId,typeOfFacility,categoryOfFacility,healthScore
0,FA0000009,RESTAURANT 0 TO 100 SEATS,BARS FRATERNAL ORGANIZATIONS,4
1,FA0000010,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,5
2,FA0000011,RESTAURANT 0 TO 100 SEATS,FAST FOOD LIMITED MENU,5
3,FA0000015,RESTAURANT 0 TO 100 SEATS,FULL SERVICE FULL MENU,3
4,FA0000017,RESTAURANT 0 TO 100 SEATS,FULL SERVICE FULL MENU,5
...,...,...,...,...
890,FA0005506,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,5
891,FA0005508,LIMITED FOOD SERVICE CONVENIENCE OTHER,FAST FOOD LIMITED MENU,5
892,FA0005510,LIMITED FOOD SERVICE CONVENIENCE OTHER,FAST FOOD LIMITED MENU,5
893,FA0005539,RESTAURANT 0 TO 100 SEATS,FAST FOOD LIMITED MENU,5


In [8]:
ave_score_df.dtypes

facilityId            object
typeOfFacility        object
categoryOfFacility    object
healthScore            int64
dtype: object

In [10]:
google_df=pd.read_csv('facility_ratings.csv', index_col=0)
google_df

,google_rating,total_ratings
facilityId,,
FA0001543,4.5,323
FA0002027,4.5,1160
FA0002162,4.1,174
FA0001624,4.3,628
FA0003535,4.3,578
...,...,...
FA0001776,4.3,541
FA0005279,4.4,1110
FA0005162,4.4,1106


In [11]:
# Add Google ratings - going to need to .join
ave_score_ratings=pd.merge(ave_score_df, google_df, on='facilityId', how='outer')
ave_score_ratings

,facilityId,typeOfFacility,categoryOfFacility,healthScore,google_rating,total_ratings
0,FA0000009,RESTAURANT 0 TO 100 SEATS,BARS FRATERNAL ORGANIZATIONS,4,4.9,15
1,FA0000010,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,5,4.7,937
2,FA0000011,RESTAURANT 0 TO 100 SEATS,FAST FOOD LIMITED MENU,5,4.3,119
3,FA0000015,RESTAURANT 0 TO 100 SEATS,FULL SERVICE FULL MENU,3,4.7,232
4,FA0000017,RESTAURANT 0 TO 100 SEATS,FULL SERVICE FULL MENU,5,4.3,109
...,...,...,...,...,...,...
890,FA0005506,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,5,4.7,422
891,FA0005508,LIMITED FOOD SERVICE CONVENIENCE OTHER,FAST FOOD LIMITED MENU,5,3.0,6
892,FA0005510,LIMITED FOOD SERVICE CONVENIENCE OTHER,FAST FOOD LIMITED MENU,5,4.9,63
893,FA0005539,RESTAURANT 0 TO 100 SEATS,FAST FOOD LIMITED MENU,5,4.8,669


In [13]:
# Bin the googleRating integers to categories "5"= 5, "4" = 4, "3" = 3, "2" = 2, "1" = 1.
bins=[0, 1, 2, 3, 4.4, 5]
google_rated = ["1", "2", "3", "4", "5"]
ave_score_ratings["googleRating"]=pd.cut(ave_score_ratings["google_rating"], bins, include_lowest=True, labels=google_rated)
ave_score_ratings

,facilityId,typeOfFacility,categoryOfFacility,healthScore,google_rating,total_ratings,googleRating
0,FA0000009,RESTAURANT 0 TO 100 SEATS,BARS FRATERNAL ORGANIZATIONS,4,4.9,15,5
1,FA0000010,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,5,4.7,937,5
2,FA0000011,RESTAURANT 0 TO 100 SEATS,FAST FOOD LIMITED MENU,5,4.3,119,4
3,FA0000015,RESTAURANT 0 TO 100 SEATS,FULL SERVICE FULL MENU,3,4.7,232,5
4,FA0000017,RESTAURANT 0 TO 100 SEATS,FULL SERVICE FULL MENU,5,4.3,109,4
...,...,...,...,...,...,...,...
890,FA0005506,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,5,4.7,422,5
891,FA0005508,LIMITED FOOD SERVICE CONVENIENCE OTHER,FAST FOOD LIMITED MENU,5,3.0,6,3
892,FA0005510,LIMITED FOOD SERVICE CONVENIENCE OTHER,FAST FOOD LIMITED MENU,5,4.9,63,5
893,FA0005539,RESTAURANT 0 TO 100 SEATS,FAST FOOD LIMITED MENU,5,4.8,669,5


In [16]:
feature_df= ave_score_ratings.drop(columns=["facilityId", "total_ratings", "google_rating"])
feature_df

,typeOfFacility,categoryOfFacility,healthScore,googleRating
0,RESTAURANT 0 TO 100 SEATS,BARS FRATERNAL ORGANIZATIONS,4,5
1,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,5,5
2,RESTAURANT 0 TO 100 SEATS,FAST FOOD LIMITED MENU,5,4
3,RESTAURANT 0 TO 100 SEATS,FULL SERVICE FULL MENU,3,5
4,RESTAURANT 0 TO 100 SEATS,FULL SERVICE FULL MENU,5,4
...,...,...,...,...
890,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,5,5
891,LIMITED FOOD SERVICE CONVENIENCE OTHER,FAST FOOD LIMITED MENU,5,3
892,LIMITED FOOD SERVICE CONVENIENCE OTHER,FAST FOOD LIMITED MENU,5,5
893,RESTAURANT 0 TO 100 SEATS,FAST FOOD LIMITED MENU,5,5


In [17]:
#ave_score_df.to_csv('average_scores.csv')

# Machine Learning Model Comparisons

In [18]:
# Preparing our data for ML
# Generate our categorical variable list
fac_cat = feature_df.dtypes[feature_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
feature_df[fac_cat].nunique()

typeOfFacility        9
categoryOfFacility    6
dtype: int64

In [19]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(feature_df[fac_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(fac_cat)
encode_df

,typeOfFacility_GROCERY STORE 0 TO 15000 SQ FT,typeOfFacility_GROCERY STORE W DELI 0 TO 15000 SQ FT,typeOfFacility_GROCERY STORE W DELI MORE THAN 15000 SQ FT,typeOfFacility_LIMITED FOOD SERVICE CONVENIENCE OTHER,typeOfFacility_MOBILE UNIT FULL FOOD SERVICE,typeOfFacility_NO FEE LICENSE K12 SCHOOLS NON PROFIT,typeOfFacility_RESTAURANT 0 TO 100 SEATS,typeOfFacility_RESTAURANT 101 TO 200 SEATS,typeOfFacility_RESTAURANT MORE THAN 200 SEATS,categoryOfFacility_BARS FRATERNAL ORGANIZATIONS,categoryOfFacility_FAST FOOD LIMITED MENU,categoryOfFacility_FULL MENU LIMITED SERVICE,categoryOfFacility_FULL SERVICE FULL MENU,categoryOfFacility_MOBILE UNITS,categoryOfFacility_RETAIL COMMISSARY
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
891,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
892,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
893,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [20]:
# Merge one-hot encoded features and drop the originals
feature_df = feature_df.merge(encode_df, left_index=True, right_index=True)
feature_df = feature_df.drop(fac_cat,1)
feature_df.head()

C:\Users\scheir3\AppData\Local\Temp/ipykernel_29032/2715996764.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  feature_df = feature_df.drop(fac_cat,1)


,healthScore,googleRating,typeOfFacility_GROCERY STORE 0 TO 15000 SQ FT,typeOfFacility_GROCERY STORE W DELI 0 TO 15000 SQ FT,typeOfFacility_GROCERY STORE W DELI MORE THAN 15000 SQ FT,typeOfFacility_LIMITED FOOD SERVICE CONVENIENCE OTHER,typeOfFacility_MOBILE UNIT FULL FOOD SERVICE,typeOfFacility_NO FEE LICENSE K12 SCHOOLS NON PROFIT,typeOfFacility_RESTAURANT 0 TO 100 SEATS,typeOfFacility_RESTAURANT 101 TO 200 SEATS,typeOfFacility_RESTAURANT MORE THAN 200 SEATS,categoryOfFacility_BARS FRATERNAL ORGANIZATIONS,categoryOfFacility_FAST FOOD LIMITED MENU,categoryOfFacility_FULL MENU LIMITED SERVICE,categoryOfFacility_FULL SERVICE FULL MENU,categoryOfFacility_MOBILE UNITS,categoryOfFacility_RETAIL COMMISSARY
0,4,5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,5,5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,5,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [21]:
# Remove Google outcome target from features data
y = feature_df.googleRating
X = feature_df.drop(columns="googleRating")

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [22]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.558


In [24]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50


UnimplementedError: Graph execution error:

Detected at node 'binary_crossentropy/Cast' defined at (most recent call last):
    File "C:\Users\scheir3\Anaconda3\envs\mlenv\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\scheir3\Anaconda3\envs\mlenv\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\scheir3\Anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\scheir3\Anaconda3\envs\mlenv\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\scheir3\Anaconda3\envs\mlenv\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\scheir3\Anaconda3\envs\mlenv\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\scheir3\Anaconda3

In [25]:
# Create the SVM model
svm = SVC(kernel='linear')

# Train the model
svm.fit(X_train, y_train)

# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")

NameError: name 'SVC' is not defined

In [ ]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

In [ ]:
# Add the prediction to the dataframe
# ave_score_df["yelpPrediction"]= y_pred
# ave_score_df.head()